# Aviation Accidents Python Analysis
## Project Goal
My project aims to determine whether the aviation business would be a good new venture for the company.
## Data Source
My data comes from [kaggle](https://www.kaggle.com/datasets/khsamaha/aviation-accident-database-synopses/data) by the NTSB aviation accident database containing information about civil aviation accidents and selected incidents within the United States and some countries.

In [2]:
# Import data analysis libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from numbers import Number
import seaborn as sns

from warnings import filterwarnings
filterwarnings("ignore")

## Read AviationData.csv.zip into a pandas DataFrame 
   I used pandas to create the DataFrame named `df`, containing the data from the dataset in file AviationData.csv.zip.

In [4]:
#Read dataset
df = pd.read_csv('./AviationData.csv.zip', encoding = 'latin1')
print(df.head())
assert type(df) == pd.DataFrame

         Event.Id Investigation.Type Accident.Number  Event.Date  \
0  20001218X45444           Accident      SEA87LA080  1948-10-24   
1  20001218X45447           Accident      LAX94LA336  1962-07-19   
2  20061025X01555           Accident      NYC07LA005  1974-08-30   
3  20001218X45448           Accident      LAX96LA321  1977-06-19   
4  20041105X01764           Accident      CHI79FA064  1979-08-02   

          Location        Country Latitude Longitude Airport.Code  \
0  MOOSE CREEK, ID  United States      NaN       NaN          NaN   
1   BRIDGEPORT, CA  United States      NaN       NaN          NaN   
2    Saltville, VA  United States  36.9222  -81.8781          NaN   
3       EUREKA, CA  United States      NaN       NaN          NaN   
4       Canton, OH  United States      NaN       NaN          NaN   

  Airport.Name  ... Purpose.of.flight Air.carrier Total.Fatal.Injuries  \
0          NaN  ...          Personal         NaN                  2.0   
1          NaN  ...         

In [7]:
# Checking dataset information and shape
print(df.info())
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

##